In [1]:
import pandas as pd

csv_file = 'torgo.csv'

# Read CSV file into pandas DataFrame
df = pd.read_csv(csv_file)

# Display the DataFrame (optional)
print(df)

                           session  \
0       F01-Session1-arrayMic-0006   
1       F01-Session1-arrayMic-0008   
2       F01-Session1-arrayMic-0009   
3       F01-Session1-arrayMic-0010   
4       F01-Session1-arrayMic-0011   
...                            ...   
16389  MC04-Session2-arrayMic-1022   
16390  MC04-Session2-arrayMic-1023   
16391  MC04-Session2-arrayMic-1024   
16392  MC04-Session2-arrayMic-1026   
16393  MC04-Session2-arrayMic-1027   

                                                   audio  \
0      /home/data1/capstone/content/downloads/Torgo//...   
1      /home/data1/capstone/content/downloads/Torgo//...   
2      /home/data1/capstone/content/downloads/Torgo//...   
3      /home/data1/capstone/content/downloads/Torgo//...   
4      /home/data1/capstone/content/downloads/Torgo//...   
...                                                  ...   
16389  /home/data1/capstone/content/downloads/Torgo//...   
16390  /home/data1/capstone/content/downloads/Torgo//...   
163

/home/jinda/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import pandas as pd
from tqdm import tqdm
import librosa

# Load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

# Load the CSV file into a DataFrame
df = pd.read_csv("torgo.csv")


# Function to process each row
def process_row(row):
    audio_path = row['audio']
    
    # Load audio and tokenize
    audio_array, sampling_rate = librosa.load(audio_path, sr=None)
    input_values = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_values
    
    # Retrieve logits
    with torch.no_grad():
        logits = model(input_values).logits
    
    # Decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Apply processing function to each row with tqdm
tqdm.pandas()
df['phoneme'] = df.progress_apply(process_row, axis=1)

# Print inference results
for idx, row in tqdm(df.iterrows(), total=len(df)):
    print(f"Audio: {row['audio']}")
    print(f"Transcription: {row['phoneme']}")
    print()
    
# Save the DataFrame with phoneme transcriptions to a CSV file
df.to_csv('torgo_phoneme.csv', index=False)

/home/jinda/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jinda/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 19%|█▊        | 3046/16394 [00:00<00:00, 15306.09it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0006.wav
Transcription: d e r t

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0008.wav
Transcription: a s ɛ ɪ n n ə i l i uː r ɑ s uː ɑ a h i v ɛ n k

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0009.wav
Transcription: p aː

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0010.wav
Transcription: aː b

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0011.wav
Transcription: m eɪ

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0012.wav
Transcription: i

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0013.wav
Transcription: n o

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0014.wav
Transcription: h iː s l oʊ k eɪ k ɐ s aɪ w ʌ ɪ n ð ɪ oʊ p ə n ɛ ɪ ɡ eɪ

Audio: /home/data1/capstone/content

 37%|███▋      | 6057/16394 [00:00<00:00, 14722.48it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0308.wav
Transcription: ɪ n f æ k t ɑːɹ h oʊ l d ᵻ f ɛ n s ɪ v j uː n ɪ t d ɪ d ɐ ɡ ʊ d dʒ ɑː b

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0309.wav
Transcription: s k ɛ t

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0310.wav
Transcription: h uː n oʊ z w ʌ t j oʊ l d ɪ s k ʌ v ɚ ɹ ɐ b aʊ t j ɔːɹ s ɛ l f

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0311.wav
Transcription: b l ɛ n d

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0312.wav
Transcription: ʃ ɛ d

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0313.wav
Transcription: ð ə f ɔːɹ b ɹ ʌ ð ɚ z l ʊ k t aɪ d ɛ n t ɪ k əl

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_headMic/0314.wav
Transcription: s aɪ d

Audio: /home/data1/capstone/content/downloads/Torgo//FC02

 55%|█████▍    | 8997/16394 [00:00<00:00, 12001.09it/s]

...

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0215.wav
Transcription: e t

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0216.wav
Transcription: m a l n i θ o n j e l o ɡ æ ŋ e m h p aː n

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0217.wav
Transcription: aɪ f eː h aɪ ɡ ə m b l ɛ n d eː v yː ɡ ɛ n

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0218.wav
Transcription: eː

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0219.wav
Transcription: b oː t i ŋ ə n w u d ɡ oː aɪ ɚ n n i n d ʌ l ʉ j

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0220.wav
Transcription: dʒ e n m eɪ ʉ n m o m ʌ n i p aɪ l ɑ w ʌ w ʌ w o d i ŋ aː

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_headMic/0221.wav
Transcription: s aɪ

Audio: /home/data1/capstone/content/downloads/Torgo//M04/Session2/wav_head

 89%|████████▊ | 14549/16394 [00:01<00:00, 13143.57it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0092.wav
Transcription: t ɔːɹ n

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0093.wav
Transcription: b ɹ iː f

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0094.wav
Transcription: ɹ eː k

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0095.wav
Transcription: ʃ ɪ p

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0096.wav
Transcription: w iː h ɐ v ɑː f t ə n ɜː dʒ d h ɪ m t ə w ɑː k m ɔːɹ ɹ æ n d s m oʊ k l ɛ s

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0097.wav
Transcription: p i k

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0098.wav
Transcription: ʃ eː

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session1/wav_arrayMic/0099.wav
Transcription: t o m

Audio: /home/data1/capstone/content/d

100%|██████████| 16394/16394 [00:01<00:00, 13273.11it/s]


Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0024.wav
Transcription: t ɹ eɪ t

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0025.wav
Transcription: ɹ eɪ n dʒ

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0026.wav
Transcription: w ɛ n h iː s p iː k s h ɪ z v oɪ s ɪ z dʒ ʌ s t ɐ b ɪ t k ɹ æ k t æ n d k w oʊ t k w ɪ v ɚ z ɐ t ɹ aɪ f əl

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0027.wav
Transcription: w ɛ n h iː s p iː k s h ɪ z v oːɹ s ɪ z dʒ ʌ s t ɐ b ɪ t k ɹ æ k t æ n d k w ɪ v ɚ z ɐ t ɹ aɪ f əl

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0028.wav
Transcription: ɡ l oʊ

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0029.wav
Transcription: f iː

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0030.wav
Transcription: ɡ ɹ æ n d f ɑː ð l ɛ k t t ə b iː m 

In [3]:
!fuser -k /dev/nvidia*

: 

: 